In [11]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

df_details = pd.read_json("/content/drive/MyDrive/netflix_Model/IMDB_movie_details.json", lines=True)
df_reviews = pd.read_json("/content/drive/MyDrive/netflix_Model/IMDB_reviews.json", lines=True)



In [17]:
df_details.describe()
df_details.head()

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...


In [27]:
df_reviews.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import spacy


In [13]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")


In [29]:
# Data preprocessing
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return ' '.join(tokens)

In [30]:
#Processed Plot Summary
df_details['processed_summary'] = df_details['plot_summary'].apply(preprocess_text)

In [ ]:
#Processed Review
df_reviews['processed_reviews'] = df_reviews['review_text'].apply(preprocess_text)

# Rating Prediction using Plot Summary


In [20]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_details['processed_summary'], df_details['rating'], test_size=0.2, random_state=42)


In [21]:
# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [22]:
# Train a simple linear regression model
model = LinearRegression()
model.fit(X_train_tfidf, y_train)


LinearRegression()

In [23]:
# Make predictions on the test set
predictions = model.predict(X_test_tfidf)


In [24]:
# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 0.9709945136108513


In [26]:
# Example usage
new_summary = "This was the craziest movie ever. Explosions, guns, rockin bods. It has everything."
new_summary = preprocess_text(new_summary)
new_summary_tfidf = vectorizer.transform([new_summary])
predicted_rating = model.predict(new_summary_tfidf)
print(f'Predicted Rating: {predicted_rating[0]}')

Predicted Rating: 6.497758126847696
